## Capture Data
The goal of this Notebook is to reorganize and place in the same folder all the Nifti files downloaded from the [ADNI dataset](http://adni.loni.usc.edu).  

When downloading the ADNI dataset in zips, each zip contains a list of folders. However, inside each folder there could be other folders or the Nifti files. In other words, the structure of all folders vary from one to another.

This notebook is structured as follows:
   - Import libraries
   - Define functions
   - Extract Nifti files

Load CT slices. https://pydicom.github.io/pydicom/dev/auto_examples/image_processing/reslice.html  
Nifti orientation issues. https://discourse.itk.org/t/nifti-orientation-issues/431  
Alzheimer propagation. https://www.memory-key.com/mynd/content/where-alzheimers-starts-and-how-it-spreads

### Import libraries

In [ ]:
import os
import gzip
import shutil

In [2]:
import nibabel as nib      # Access to neuroimaging data formats
import med2image           # Convert medical images to jpg and png (pip install med2image)
import matplotlib.pyplot as plt
import cv2
from random import randint
import numpy as np
from aux_functions.aux_functions_cleaning import *

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from deepbrain import Extractor

### Define functions

In [ ]:
def extract_images(root, new_root):
    '''
    Function to extract nifti images from a folder (root), zip them and save them in new fodler (new_root)
    ''' 
    
    # Create new directory to move files in case it doesn´t exist
    if not os.path.exists(new_root):
        os.makedirs(new_root)

    # Get directions of the file and move them to new folder
    for folder in sorted(os.listdir(root)):

        # Avoid trigerring .DS_Store
        if folder.startswith('.'):
            continue

        directions = []
        directions.append(os.path.join(root,folder))
        all_files = False

        while all_files == False:
            for index, path in enumerate(directions):

                if (os.path.isfile(path)) == False:

                    for subfolder in sorted(os.listdir(path)):

                        if subfolder.startswith('.'):
                            continue

                        directions.append(os.path.join(path,subfolder))

                    directions.remove(path)

                if index == (len(directions) -1):
                    is_not_file = False

                    for item in directions:
                        if (os.path.isfile(item)) == True:
                            continue
                        else:
                            is_not_file = True

                    if is_not_file == False:
                        all_files = True
                    else:
                        break
    
        # Copy files, compress them and move them to new folder
        for direction in directions:
            new_direction = os.path.join(new_root,direction.split("/")[-1]) + ".gz"

            # Check if file already exist in the new folder
            if os.path.exists(new_direction) == False:
                
                # Copy image file to new folder
                with open(direction, 'rb') as f_in:
                    with gzip.open(new_direction, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
            else:
                print(f"File already exists in the new folder: {new_direction}")
              
        print(folder)
        print("*" * 30)

### Nifti files extraction
Extract nifti files from multiple folders (root), zip them and save them in a new folder (new root).

#### Specify paths

In [ ]:
# Define root path where the zips downloaded from ADNI dataset are
root = "ADNI1_Complete_3Yr_1.5T"
print(f"Number of files on {root} folder =",len(os.listdir(root)))

# Define the folder path where to move the Nifti files zipped
new_root = "ADNI1_Complete_3Yr_1.5T_images" 

#### Run process

In [ ]:
# Run process - extract Nifti files from folder root and move them to folder new_root
extract_images(root, new_root)